<a href="https://colab.research.google.com/github/Logeshwari-2001/Anomaly/blob/main/svm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
data = pd.read_csv('/content/Train_data.csv')


In [3]:
# Separate the target variable and the feature variables
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [10]:
print(X["protocol_type"])

0        tcp
1        udp
2        tcp
3        tcp
4        tcp
        ... 
25187    tcp
25188    tcp
25189    tcp
25190    tcp
25191    tcp
Name: protocol_type, Length: 25192, dtype: object


In [25]:
print(X)
print(X.shape)

       duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0             0              0        0     0        491          0     0   
1             0              1        1     0        146          0     0   
2             0              0        2     1          0          0     0   
3             0              0        3     0        232       8153     0   
4             0              0        3     0        199        420     0   
...         ...            ...      ...   ...        ...        ...   ...   
25187         0              0       43     5          0          0     0   
25188         0              0        0     0        334          0     0   
25189         0              0        2     2          0          0     0   
25190         0              0       54     1          0          0     0   
25191         0              0       10     1          0          0     0   

       wrong_fragment  urgent  hot  ...  dst_host_count  dst_host_srv_count

In [6]:
print(y)

0         normal
1         normal
2        anomaly
3         normal
4         normal
          ...   
25187    anomaly
25188    anomaly
25189    anomaly
25190    anomaly
25191    anomaly
Name: class, Length: 25192, dtype: object


In [ ]:
print(X[protocol_type])

In [13]:
# Convert non-numeric features to numeric
X["protocol_type"] = pd.factorize(X["protocol_type"])[0]
X["service"] = pd.factorize(X["service"])[0]
X["flag"] = pd.factorize(X["flag"])[0]

In [12]:
print(X["protocol_type"])

0        0
1        1
2        0
3        0
4        0
        ..
25187    0
25188    0
25189    0
25190    0
25191    0
Name: protocol_type, Length: 25192, dtype: int64


In [14]:
# Apply standard scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
from sklearn.ensemble import RandomForestClassifier
rfe = RFE(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1), n_features_to_select = 15)
rfe.fit(X_scaled, y)

RFE(estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
    n_features_to_select=15)

In [18]:
feature_names = list(X.columns[rfe.support_])
print(feature_names)

['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'hot', 'logged_in', 'count', 'srv_count', 'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate']


In [19]:
with open('scaler1.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('rfe.pkl', 'wb') as f:
    pickle.dump(rfe, f)

In [28]:
names = data.columns[:-1]

In [30]:
selected_features = rfe.support_

In [31]:
X_selected = X_scaled[:, selected_features]

In [38]:
svm_model = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42).fit(X_selected, y)

In [29]:
from sklearn.svm import SVC

# X_train and y_train are the preprocessed feature matrix and target vector, respectively
# svm_model = SVC(kernel='rbf', C=10, gamma=0.1, random_state=42)
# svm_model = SVC(kernel='linear', C=1).fit(X_scaled[:, feature_names], y, names=names[feature_names])
# svm_model.fit(X_scaled, y)

IndexError: ignored

In [41]:
with open('svm1.pkl', 'wb') as f:
    pickle.dump(svm_model, f)

In [39]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
X_train_prediction = svm_model.predict(X_selected)
training_data_accuracy = accuracy_score(y, X_train_prediction)
print(training_data_accuracy)

0.9916243251825977
